In [ ]:
import appsettings

In [ ]:
SERVER = appsettings.SERVER
USER = appsettings.USER
PASSWORD = appsettings.PASSWORD
DRIVER = '{Amazon Redshift ODBC Driver (x64)}'
DATABASE = 'dev'
SCHEMA = 'public'
ROWS = 10000
LEVEL = 't'
TABLE = 'category'
ASSOCIATIONS = False
OPEN_BROWSER = False

In [ ]:
import os
import pyodbc
import pandas as pd
import sweetviz as sv
import gc

In [ ]:
def create_connection():
    return pyodbc.connect('DRIVER='+DRIVER+';SERVER='+SERVER+';PORT=5439;DATABASE='+DATABASE+';UID='+USER+';PWD='+PASSWORD)

In [ ]:
tables = []

if(LEVEL=='s'):
    with create_connection() as conn1:
        with conn1.cursor() as schema_cur:
            schema_cur.execute('select "table", size, pct_used, tbl_rows from svv_table_info')
            schema_row = schema_cur.fetchone()
            while schema_row:
                table_name = str(schema_row[0])
                table_size = str(schema_row[1])
                table_pct_used = str(schema_row[2])
                table_rows = str(schema_row[3])

                tables.append((f'{SCHEMA}.{table_name}',table_size,table_pct_used,table_rows))
                schema_row = schema_cur.fetchone()
else:
    with create_connection() as conn1:
        with conn1.cursor() as table_cur:
            table_cur.execute(f"select size, pct_used, tbl_rows from svv_table_info where svv_table_info.table='{TABLE}'")
            table_row = table_cur.fetchone()
            
            table_size = str(table_row[0])
            table_pct_used = str(table_row[1])
            table_rows = str(table_row[2])

            tables.append((f'{SCHEMA}.{TABLE}',table_size,table_pct_used,table_rows))

In [ ]:
if not os.path.exists('obj'):
    os.mkdir('obj')

info_arr = []

for (table_name,table_size,table_pct_used,table_rows) in tables:
    print(f'Loading {table_name}...')

    sheet_name = table_name.split('.')[-1]

    sample_query = f'SELECT * FROM {table_name} ORDER BY RANDOM() LIMIT {ROWS}'
    with create_connection() as conn3:
        sample_cur = conn3.cursor().execute(sample_query)
        sample_df = pd.DataFrame.from_records(
            iter(sample_cur), columns=[x[0] for x in sample_cur.description])

        info_arr.append([table_name,table_size,table_pct_used,table_rows, len(sample_df)])

        analysis = sv.analyze(sample_df, pairwise_analysis=("on" if ASSOCIATIONS else "off"))
        analysis.show_html(f'obj/{sheet_name}.html', open_browser=OPEN_BROWSER)

    del sample_df
    gc.collect()

In [ ]:
eda_info_df = pd.DataFrame(info_arr, columns = ['TABLE NAME','TABLE SIZE (MB)','TABLE PCT. USED','TABLE ROWS','SAMPLE ROWS'])
excel_writer = pd.ExcelWriter(f'obj/{SCHEMA}_EDA_INFO.xlsx', engine='xlsxwriter')
eda_info_df.to_excel(excel_writer, sheet_name=SCHEMA, index=False)
worksheet = excel_writer.sheets[SCHEMA]
for idx, col in enumerate(eda_info_df):  # Loop through all columns
    series = eda_info_df[col]
    max_len = max((
        series.astype(str).str.len().max(),  # Len of largest item
        len(str(series.name))  # Len of column name/header
        )) + 9  # Adding a little extra space
    worksheet.set_column(idx, idx, max_len)  # Set column width
excel_writer.close()